In [ ]:
!git clone https://github.com/Xnehil/ZmBART.git

In [4]:
import os
os.chdir("ZmBART") 

#!git clone https://gist.github.com/f959c19ec18fee3ce2ff9b3b86b67c16.git
#!mv /kaggle/working/ZmBART/f959c19ec18fee3ce2ff9b3b86b67c16/convert_envyml_to_reqtxt /kaggle/working/ZmBART
#!python /kaggle/working/ZmBART/convert_envyml_to_reqtxt

In [ ]:
mkdir -p ~/miniconda3
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm ~/miniconda3/miniconda.sh

~/miniconda3/bin/conda init bash
~/miniconda3/bin/conda init zsh

In [ ]:
!wget https://dl.fbaipublicfiles.com/fairseq/models/mbart/mbart.CC25.tar.gz
!tar -xzvf mbart.CC25.tar.gz

In [ ]:
conda env create --file environment.yml
conda activate py37_ZmBART

In [ ]:
import subprocess
import numpy as np

# Define variables
DATA = "../data"
TRAIN = "train"
VALID = "valid"
TEST = "test"
SRC = "isk"
TGT = "isk"
NAME = "isk-isk"
DEST = "../dataset/postprocess/aux"
DICT = "/workspace/mbart.cc25/dict.txt"

# Construct the command
command = (
    f"python -u preprocess.py "
    f"--source-lang {SRC} "
    f"--target-lang {TGT} "
    f"--trainpref {DATA}/{TRAIN}.spm "
    f"--validpref {DATA}/{VALID}.spm "
    f"--testpref {DATA}/{TEST}.spm "
    f"--destdir {DEST}/{NAME} "
    f"--thresholdtgt 0 "
    f"--thresholdsrc 0 "
    f"--srcdict {DICT} "
    f"--tgtdict {DICT} "
    f"--workers 70 "
    f"--fp16"
)

# Execute the command
subprocess.run(command, shell=True)

In [ ]:
!pip uninstall torch torchvision torchaudio -y
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118

In [ ]:
#Torch version
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.version.cuda)  # Should return 11.8
print(torch.cuda.get_device_name(0))  # Should return NVIDIA RTX A4000


In [7]:
#Pretrain
import subprocess
import numpy as np

# Define the variables
PRETRAIN = "/workspace/mbart.cc25"
langs = "ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN"
SRC = "isk"
TGT = "isk"
NAME = "isk-isk"
DATADIR = "../dataset/postprocess/aux/isk-isk"
SAVEDIR = "/workspace/checkpoint_ZmBART"

# Construct the command
command = [
    "python", "-u", "train.py", DATADIR,
    "--arch", "mbart_large",
    "--task", "translation_from_pretrained_bart",
    "--source-lang", SRC,
    "--target-lang", TGT,
    "--criterion", "label_smoothed_cross_entropy",
    "--label-smoothing", "0.2",
    "--optimizer", "adam",
    "--adam-eps", "1e-06",
    "--lr-scheduler", "polynomial_decay",
    "--lr", "3e-05",
    "--warmup-updates", "2500",
    "--max-update", "100000",
    "--dropout", "0.3",
    "--max-tokens", "2048",
    "--update-freq", "2",
    "--save-interval", "6",
    "--save-interval-updates", "50000",
    "--keep-interval-updates", "4",
    "--seed", "222",
    "--log-interval", "100",
    "--restore-file", SAVEDIR,
    "--langs", langs,
    "--save-dir", SAVEDIR,
    "--skip-invalid-size-inputs-valid-test"
]

# Monkey patch np.float to np.float64
# np.float = np.float64
print("Running command:", " ".join(command))

# Monkey patch np.float to np.float64
np.float = np.float64

# Run the command with error handling
try:
    result = subprocess.run(command, check=True, capture_output=True, text=True)
    print("Command output:", result.stdout)
    print("Command error (if any):", result.stderr)
except subprocess.CalledProcessError as e:
    print("An error occurred while running the command.")
    print("Return code:", e.returncode)
    print("Output:", e.output)
    print("Error:", e.stderr)

Running command: python -u train.py ../dataset/postprocess/aux/isk-isk --arch mbart_large --task translation_from_pretrained_bart --source-lang isk --target-lang isk --criterion label_smoothed_cross_entropy --label-smoothing 0.2 --optimizer adam --adam-eps 1e-06 --lr-scheduler polynomial_decay --lr 3e-05 --warmup-updates 2500 --max-update 100000 --dropout 0.3 --max-tokens 2048 --update-freq 2 --save-interval 6 --save-interval-updates 50000 --keep-interval-updates 4 --seed 222 --log-interval 100 --restore-file /workspace/checkpoint_ZmBART --langs ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN --save-dir /workspace/checkpoint_ZmBART --skip-invalid-size-inputs-valid-test


KeyboardInterrupt: 